In [7]:
import whisper
from pydub import AudioSegment
import os
import re
import urllib.parse

# 去除非法字符，确保文件名合法
def sanitize_filename(text):
    # 使用 URL 编码来确保中文字符不会导致文件名问题
    return urllib.parse.quote(text)

# 处理单个文件的音频分段
def segment_audio_with_timestamps(audio_file_path, output_folder="output_segments"):
    # 加载 Whisper 模型
    print(f"加载 Whisper 模型...")
    model = whisper.load_model("large")

    # 转录音频并获取时间戳
    print(f"转录音频文件: {audio_file_path}...")
    result = model.transcribe(audio_file_path, word_timestamps=True)

    # 加载音频文件
    audio = AudioSegment.from_wav(audio_file_path)
    print(f"加载音频文件: {audio_file_path}")

    # 创建输出文件夹，如果不存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 遍历每个字幕段并保存相应的音频片段
    for idx, segment in enumerate(result['segments']):
        start_time_ms = int(segment["start"] * 1000)  # 转换为毫秒
        end_time_ms = int(segment["end"] * 1000)  # 转换为毫秒

        # 切割音频段
        audio_segment = audio[start_time_ms:end_time_ms]

        # 使用字幕内容作为文件名，进行 URL 编码处理以支持繁体中文字符
        subtitle_text = segment["text"].strip()
        if subtitle_text:  # 如果字幕非空
            # 去除非法字符并使用 URL 编码以确保文件名合法
            sanitized_name = sanitize_filename(subtitle_text) + ".wav"
            output_filename = os.path.join(output_folder, f"segment_{idx + 1}_{sanitized_name}")
        else:
            # 如果字幕为空，使用默认命名
            output_filename = os.path.join(output_folder, f"segment_{idx + 1}.wav")

        # 保存音频段到文件
        audio_segment.export(output_filename, format="wav")
        print(f"保存片段 {idx + 1} 到 {output_filename}")

    print(f"音频分段完成，已保存 {len(result['segments'])} 个片段到 {output_folder}。")

# 处理多个音频文件的主函数
def process_multiple_files(input_folder, output_folder="output_segments"):
    # 获取文件夹中的所有 .wav 文件
    wav_files = [f for f in os.listdir(input_folder) if f.endswith(".wav")]
    
    # 如果没有找到 wav 文件
    if not wav_files:
        print(f"在文件夹 {input_folder} 中没有找到任何 .wav 文件。")
        return

    print(f"开始处理文件夹中的 {len(wav_files)} 个音频文件...")

    # 遍历所有音频文件并调用分段处理函数
    for wav_file in wav_files:
        audio_file_path = os.path.join(input_folder, wav_file)
        print(f"\n正在处理文件: {wav_file}")
        segment_audio_with_timestamps(audio_file_path, output_folder)

# 主函数
def main():
    # 输入文件夹路径，包含所有的 .wav 文件
    input_folder = "/home/victor/AudioLDM-training-finetuning/data/Selected_audio"  # 替换为你自己的文件夹路径

    # 输出文件夹路径，保存切割后的音频片段
    output_folder = "output_segments"  # 你可以修改这个输出文件夹

    # 处理文件夹中的所有音频文件
    process_multiple_files(input_folder, output_folder)

# 运行主函数
if __name__ == "__main__":
    main()


开始处理文件夹中的 34 个音频文件...

正在处理文件: 23.s「智醒．智慧醫療」第一堂：智慧醫療與日常生活的關係YF.wav
加载 Whisper 模型...


RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
